In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, CommaSeparatedListOutputParser, StrOutputParser
import boto3
import os
import re
import json
from datetime import datetime
from langchain_core.pydantic_v1 import Field
from pydantic import BaseModel
from time_converter import TimeConverter
import kss
load_dotenv()

/Users/mac/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


True

In [2]:
subject = '생명과학1'
room_id = '675670d63ece5b11778911a0'

s3 = boto3.client('s3')

bucket_name = 'pagecall-text'
parent_folder = f'{subject}/{room_id}'

def get_items(bucket, folder):
    response = s3.list_objects_v2(Bucket=bucket, Prefix=folder)
    content_list = []
    for content in response.get('Contents', []):
        key = content.get('Key', None)
        if len(key.split('/')) == 4:
            content_list.append(key)
    return content_list

def download_items(bucket, file_keys, local_dir):
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)
    
    for file_key in file_keys:
        folder_name = os.path.basename(os.path.dirname(file_key))
        file_name = os.path.basename(file_key)
        local_file_path = os.path.join(local_dir, f"{folder_name}_{file_name}")
        s3.download_file(bucket, file_key, local_file_path)
        
file_keys = get_items(bucket_name, parent_folder)
download_items(bucket_name, file_keys, 'downloads')

In [3]:
local_download_dir = './downloads'

def merge_files(directory):
    merged_data = []
    
    for file_name in os.listdir(directory):
        parts = file_name.split('_')
        absolute_start_time = int(parts[-2])
        speaker = 'teacher' if 'T' in parts[0] else 'student'
    
        file_path = os.path.join(directory, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = json.load(f)
            
            for item in content:
                start_ms = TimeConverter.convert_time_to_milliseconds(item['start'])
                end_ms = TimeConverter.convert_time_to_milliseconds(item['end'])
                
                merged_item = {
                    "absolute_start_time": absolute_start_time + start_ms,
                    "absolute_end_time": absolute_start_time + end_ms,
                    "time_range": None,
                    "speaker": speaker,
                    "text": item["text"].strip(),
                }
                merged_data.append(merged_item)
                
    # 절대 시작 시간을 기준으로 정렬
    merged_data.sort(key=lambda x: x["absolute_start_time"])

    global_start_time = merged_data[0]["absolute_start_time"]  # 가장 이른 절대 시작 시간
    for item in merged_data:
        item["time_range"] = f"{TimeConverter.format_ms_to_xm_ys(item['absolute_start_time'] - global_start_time)} ~ {TimeConverter.format_ms_to_xm_ys(item['absolute_end_time'] - global_start_time)}"
        item['absolute_start_time'] = TimeConverter.format_ms_to_ktc(item['absolute_start_time'])
        item['absolute_end_time'] = TimeConverter.format_ms_to_ktc(item['absolute_end_time'])
    
    return merged_data

In [4]:
raw_data = merge_files(local_download_dir)

def extract_data(is_time, speaker=None):
    if (is_time) & (speaker == None):
        return [{'time_range': item['time_range'], 'speaker': item['speaker'], 'text': item['text']} for item in raw_data]
    if (is_time) & (speaker == 'teacher'):
        return [{'time_range': item['time_range'], 'speaker': item['speaker'], 'text': item['text']} for item in raw_data if item['speaker'] == 'teacher']
    if (is_time) & (speaker == 'student'):
        return [{'time_range': item['time_range'], 'speaker': item['speaker'], 'text': item['text']} for item in raw_data if item['speaker'] == 'student']
    if (is_time==False) & (speaker == 'teacher'):
        return ' '.join([item['text'] for item in raw_data if item['speaker'] == 'teacher'])
    if (is_time==False) & (speaker == 'student'):
        return ' '.join([item['text'] for item in raw_data if item['speaker'] == 'student'])

extracted_data = extract_data(is_time=False, speaker='teacher')
extracted_data

'안녕하세요. 안녕하세요. 소리 잘 들리나요? 네. 그럼 저희 이제 생명과학 수업 시작할 건데 수업 준비가 되셨을까요? 네. 네. 그럼 수업 시작해 볼게요. 오늘은 첫 수업이라서 간단하게 OT 진행하고 고2 10월 모의고사 풀어보라고 했었죠? 네. 네. 그거 몇 문제 같이 풀어보고 남는 시간에 그 세포 분열 수업 잠깐 진행할게요. 네 그 교재는 그 마더통 검은색 책 2026거 출판되서 그거 구매하기로 했었죠 그 혹시 언제 교재에 살 수 있을까요? 어제 구매했어요? 아 그러면 교재 한 다음주 전까지만 빨리 등록해주세요 검은색으로 사신거죠? 네 그리고 저희 그럼 OT 간단하게 진행할게요. 수업 시작하기 전에 질문 관련해서 드릴 말씀이 있는데 혹시 가현 학생이 평소에 질문 잘하시는 편인가요? 질문 잘하시는 편인 거네요. 저도 학생일 때는 질문하는 걸 좀 어려워 했었거든요. 1대1 수업이나 아니면 선생님 따로 찾아가지 않는 이상은 질문하기 쉽지 않잖아요. 그래도 이제 치룸들여서 혼자 고민하다 보면 답 나오기도 하니까 질문 많이 하려고 하진 않았었는데 저는 그래도 이제 저희는 1대1 수업이잖아요. 과의 장점이 학생 한 명한테 맞춰서 수업 진행할 수 있고 서로 의사소통 된다는 게 장점이잖아요. 그래서 가윤 학생이 저한테 언제든지 편하게 질문을 해주셨으면 좋겠거든요. 저를 좀 어렵게 생각하지 마시고 의사소통할 때도 너무 긴장하거나 하지 않고 그냥 편하게 말씀해주셨으면 좋겠어요. 저랑 질문하면서 빠르게 이해할 수 있으면 더 좋은 거니까 더 오래 남기도 하고 모르는 게 무엇인지 구체적으로 질문해주세요. 그러면 이제 제 소개 잠깐 간단하게 해볼게요. 일단 알고 계시겠지만 제 이름은 정예온이고요. 저는 지금 이대 약학과에 재학하고 있어요. 그리고 저는 2023년도에 수능 보고 정시로 대학을 입학했거든요. 그래서 저는 입시 치르면서 독서실 형태의 학원도 다녀본 적 있고 시대인지 아시나요? 시대인지 학원 혹시 아시나요? 아 그래요? 그 대치동에 있는 학원 있는데 거기 재수종합반 다녀

In [5]:
splited_data = []
for idx, sent in enumerate(kss.split_sentences(extracted_data)):
    splited_data.append({'idx': idx, 'text': sent})
splited_data

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



[{'idx': 0, 'text': '안녕하세요.'},
 {'idx': 1, 'text': '안녕하세요.'},
 {'idx': 2, 'text': '소리 잘 들리나요?'},
 {'idx': 3, 'text': '네.'},
 {'idx': 4, 'text': '그럼 저희 이제 생명과학 수업 시작할 건데 수업 준비가 되셨을까요?'},
 {'idx': 5, 'text': '네. 네. 그럼 수업 시작해 볼게요.'},
 {'idx': 6, 'text': '오늘은 첫 수업이라서 간단하게 OT 진행하고 고2 10월 모의고사 풀어보라고 했었죠?'},
 {'idx': 7, 'text': '네. 네. 그거 몇 문제 같이 풀어보고 남는 시간에 그 세포 분열 수업 잠깐 진행할게요.'},
 {'idx': 8, 'text': '네 그 교재는 그 마더통 검은색 책 2026거 출판되서 그거 구매하기로 했었죠'},
 {'idx': 9, 'text': '그 혹시 언제 교재에 살 수 있을까요?'},
 {'idx': 10, 'text': '어제 구매했어요?'},
 {'idx': 11, 'text': '아 그러면 교재 한 다음주 전까지만 빨리 등록해주세요'},
 {'idx': 12, 'text': '검은색으로 사신거죠?'},
 {'idx': 13, 'text': '네 그리고 저희 그럼 OT 간단하게 진행할게요.'},
 {'idx': 14,
  'text': '수업 시작하기 전에 질문 관련해서 드릴 말씀이 있는데 혹시 가현 학생이 평소에 질문 잘하시는 편인가요?'},
 {'idx': 15, 'text': '질문 잘하시는 편인 거네요.'},
 {'idx': 16, 'text': '저도 학생일 때는 질문하는 걸 좀 어려워 했었거든요.'},
 {'idx': 17, 'text': '1대1 수업이나 아니면 선생님 따로 찾아가지 않는 이상은 질문하기 쉽지 않잖아요.'},
 {'idx': 18,
  'text': '그래도 이제 치룸들여서 혼자 고민하다 보면 답 나오기도 하니까 질문 많이 하려고 하진 않았었는데 

In [6]:
def split_with_overlap(data, chunk_size, overlap):
    result = []
    step = chunk_size - overlap
    for i in range(0, len(data), step):
        result.append(data[i:i + chunk_size])
        if i + chunk_size >= len(data):
            break
    return result

chunk_size = 30
overlap = 5
chunks_with_overlap = split_with_overlap(splited_data, chunk_size, overlap)

In [8]:
from langchain_core.runnables import RunnableParallel
from prompt.question_analyzer import QuestionAnalyzer

llm = ChatOpenAI(model='gpt-4o')

system_prompt = QuestionAnalyzer(subject).prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
question_analysis_results = chain.batch([
    {"user_message": chunk} for chunk in chunks_with_overlap
])

In [9]:
def extract_indices(data):
    all_indices = []
    for entry in data:
        json_text = re.search(r'```json\n(.*?)\n```', entry, re.DOTALL).group(1)
        parsed_json = json.loads(json_text)
        all_indices.extend(parsed_json[0]["idx"])
    return sorted(set(all_indices))

question_indices = extract_indices(question_analysis_results)
question_indices

[2,
 4,
 6,
 9,
 10,
 12,
 14,
 27,
 28,
 29,
 32,
 33,
 41,
 48,
 49,
 50,
 51,
 53,
 54,
 55,
 56,
 57,
 59,
 61,
 62,
 65,
 66,
 69,
 71,
 76,
 81,
 82,
 92,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 105,
 106,
 109,
 114,
 123,
 124,
 128,
 129,
 130,
 131,
 135,
 136,
 137,
 139,
 143,
 146,
 147,
 148,
 150,
 153,
 155,
 156,
 159,
 160,
 163,
 164,
 169,
 170,
 171,
 176,
 177,
 179,
 180,
 186,
 190,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 212,
 213,
 214,
 218,
 219,
 227,
 228,
 229,
 230,
 231,
 234,
 240,
 241,
 243,
 244,
 247,
 250,
 251,
 254,
 267,
 268,
 269,
 275,
 277,
 278,
 279,
 281,
 282,
 283,
 284,
 286,
 287,
 289,
 292,
 293,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 304,
 315,
 318,
 322,
 323,
 325,
 326,
 327,
 328,
 332,
 333,
 336,
 338,
 342,
 343,
 345,
 348,
 349,
 352,
 353,
 356,
 363,
 367,
 368,
 369,
 371,
 372,
 375,
 378,
 382,
 391,
 394,
 403,
 405,
 406,
 409,
 415,
 419,
 420,
 421,
 422,
 425,
 426,
 427,
 428,
 429,
 432,
 433,

In [10]:
[item for item in splited_data if item['idx'] in question_indices]

[{'idx': 2, 'text': '소리 잘 들리나요?'},
 {'idx': 4, 'text': '그럼 저희 이제 생명과학 수업 시작할 건데 수업 준비가 되셨을까요?'},
 {'idx': 6, 'text': '오늘은 첫 수업이라서 간단하게 OT 진행하고 고2 10월 모의고사 풀어보라고 했었죠?'},
 {'idx': 9, 'text': '그 혹시 언제 교재에 살 수 있을까요?'},
 {'idx': 10, 'text': '어제 구매했어요?'},
 {'idx': 12, 'text': '검은색으로 사신거죠?'},
 {'idx': 14,
  'text': '수업 시작하기 전에 질문 관련해서 드릴 말씀이 있는데 혹시 가현 학생이 평소에 질문 잘하시는 편인가요?'},
 {'idx': 27, 'text': '그래서 저는 입시 치르면서 독서실 형태의 학원도 다녀본 적 있고 시대인지 아시나요?'},
 {'idx': 28, 'text': '시대인지 학원 혹시 아시나요?'},
 {'idx': 29, 'text': '아 그래요?'},
 {'idx': 32, 'text': '경기도 하세요?'},
 {'idx': 33, 'text': '경기도 수원?'},
 {'idx': 41, 'text': '일단 여기까지 자기소개 간단하기만 했는데 혹시 저한테 더 궁금한 점 있나요?'},
 {'idx': 48, 'text': '네 가윤 학생 혹시 수시예요'},
 {'idx': 49, 'text': '정시예요?'},
 {'idx': 50, 'text': '아직 정해진 거 없나요?'},
 {'idx': 51, 'text': '네 아 그럼 수시 생각하고 최저 맞추는 거 생각하고 있는 건가요?'},
 {'idx': 53, 'text': '또 궁금한 거 있나요?'},
 {'idx': 54, 'text': '없애도 있나요?'},
 {'idx': 55, 'text': '없어요?'},
 {'idx': 56, 'text': '없어요?'},
 {'idx': 57, 'text': '아니면 뭐 가현 학생 또 저한테 소개

In [11]:
def get_grouped_surrounding_texts(data, target_indices, range_size=3):
    grouped_result = []
    for idx in target_indices:
        target_sentence = data[idx]['text']
        start_idx = max(0, idx - range_size)
        end_idx = min(len(data), idx + range_size + 1)
        contexts = data[start_idx:end_idx]
        context = ' '.join([item['text'] for item in contexts])
        grouped_result.append({'idx': idx, 'target_sentence': target_sentence, 'context': context})
    return grouped_result

# 결과 출력
grouped_surrounding_texts = get_grouped_surrounding_texts(splited_data, question_indices)
grouped_surrounding_texts

[{'idx': 2,
  'target_sentence': '소리 잘 들리나요?',
  'context': '안녕하세요. 안녕하세요. 소리 잘 들리나요? 네. 그럼 저희 이제 생명과학 수업 시작할 건데 수업 준비가 되셨을까요? 네. 네. 그럼 수업 시작해 볼게요.'},
 {'idx': 4,
  'target_sentence': '그럼 저희 이제 생명과학 수업 시작할 건데 수업 준비가 되셨을까요?',
  'context': '안녕하세요. 소리 잘 들리나요? 네. 그럼 저희 이제 생명과학 수업 시작할 건데 수업 준비가 되셨을까요? 네. 네. 그럼 수업 시작해 볼게요. 오늘은 첫 수업이라서 간단하게 OT 진행하고 고2 10월 모의고사 풀어보라고 했었죠? 네. 네. 그거 몇 문제 같이 풀어보고 남는 시간에 그 세포 분열 수업 잠깐 진행할게요.'},
 {'idx': 6,
  'target_sentence': '오늘은 첫 수업이라서 간단하게 OT 진행하고 고2 10월 모의고사 풀어보라고 했었죠?',
  'context': '네. 그럼 저희 이제 생명과학 수업 시작할 건데 수업 준비가 되셨을까요? 네. 네. 그럼 수업 시작해 볼게요. 오늘은 첫 수업이라서 간단하게 OT 진행하고 고2 10월 모의고사 풀어보라고 했었죠? 네. 네. 그거 몇 문제 같이 풀어보고 남는 시간에 그 세포 분열 수업 잠깐 진행할게요. 네 그 교재는 그 마더통 검은색 책 2026거 출판되서 그거 구매하기로 했었죠 그 혹시 언제 교재에 살 수 있을까요?'},
 {'idx': 9,
  'target_sentence': '그 혹시 언제 교재에 살 수 있을까요?',
  'context': '오늘은 첫 수업이라서 간단하게 OT 진행하고 고2 10월 모의고사 풀어보라고 했었죠? 네. 네. 그거 몇 문제 같이 풀어보고 남는 시간에 그 세포 분열 수업 잠깐 진행할게요. 네 그 교재는 그 마더통 검은색 책 2026거 출판되서 그거 구매하기로 했었죠 그 혹시 언제 교재에 살 수 있을까요? 어제 구매했어요? 아 그러면 교

In [12]:
from langchain_core.runnables import RunnableParallel
from prompt.question_classifier import QuestionClassifier

llm = ChatOpenAI(model='gpt-4o')

system_prompt = QuestionClassifier(subject).prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

question_classifier_results = chain.batch([
    {"user_message": chunk} for chunk in grouped_surrounding_texts
])

In [15]:
for (res,item) in zip(question_classifier_results, grouped_surrounding_texts):
    item['result'] = res
grouped_surrounding_texts

[{'idx': 2,
  'target_sentence': '소리 잘 들리나요?',
  'context': '안녕하세요. 안녕하세요. 소리 잘 들리나요? 네. 그럼 저희 이제 생명과학 수업 시작할 건데 수업 준비가 되셨을까요? 네. 네. 그럼 수업 시작해 볼게요.',
  'result': 'False'},
 {'idx': 4,
  'target_sentence': '그럼 저희 이제 생명과학 수업 시작할 건데 수업 준비가 되셨을까요?',
  'context': '안녕하세요. 소리 잘 들리나요? 네. 그럼 저희 이제 생명과학 수업 시작할 건데 수업 준비가 되셨을까요? 네. 네. 그럼 수업 시작해 볼게요. 오늘은 첫 수업이라서 간단하게 OT 진행하고 고2 10월 모의고사 풀어보라고 했었죠? 네. 네. 그거 몇 문제 같이 풀어보고 남는 시간에 그 세포 분열 수업 잠깐 진행할게요.',
  'result': 'False'},
 {'idx': 6,
  'target_sentence': '오늘은 첫 수업이라서 간단하게 OT 진행하고 고2 10월 모의고사 풀어보라고 했었죠?',
  'context': '네. 그럼 저희 이제 생명과학 수업 시작할 건데 수업 준비가 되셨을까요? 네. 네. 그럼 수업 시작해 볼게요. 오늘은 첫 수업이라서 간단하게 OT 진행하고 고2 10월 모의고사 풀어보라고 했었죠? 네. 네. 그거 몇 문제 같이 풀어보고 남는 시간에 그 세포 분열 수업 잠깐 진행할게요. 네 그 교재는 그 마더통 검은색 책 2026거 출판되서 그거 구매하기로 했었죠 그 혹시 언제 교재에 살 수 있을까요?',
  'result': 'False'},
 {'idx': 9,
  'target_sentence': '그 혹시 언제 교재에 살 수 있을까요?',
  'context': '오늘은 첫 수업이라서 간단하게 OT 진행하고 고2 10월 모의고사 풀어보라고 했었죠? 네. 네. 그거 몇 문제 같이 풀어보고 남는 시간에 그 세포 분열 수업 잠깐 진행할게요. 네 그 교재는 그 마더통 검은색 책 

In [8]:
from langchain_core.runnables import RunnableParallel
from prompt.question_analyzer_advanced import QuestionAnalyzerAdvanced

llm = ChatOpenAI(model='gpt-4o')

system_prompt = QuestionAnalyzerAdvanced(subject).prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
question_advanced_analysis_results = chain.batch([
    {"user_message": chunk} for chunk in chunks_with_overlap
])

In [9]:
import json
import re

# 학습질문 idx와 일반질문 idx 추출 함수
def extract_indices(data):
    extracted_data = []
    for entry in data:
        # JSON 부분만 추출
        json_text = re.search(r'```json\n(.*?)\n```', entry, re.DOTALL).group(1)
        # JSON 파싱
        parsed_json = json.loads(json_text)
        result = {
            "학습질문 idx": parsed_json[0]["학습질문 idx"],
            "일반질문 idx": parsed_json[1]["일반질문 idx"]
        }
        extracted_data.append(result)
    return extracted_data

temp = extract_indices(question_advanced_analysis_results)
learning_indices = sorted(set(sum([entry["학습질문 idx"] for entry in temp], [])))
general_indices = sorted(set(sum([entry["일반질문 idx"] for entry in temp], [])))

print('학습질문 : ', learning_indices)
print('일반질문 : ', general_indices)

학습질문 :  [96, 97, 105, 124, 128, 129, 135, 146, 147, 148, 155, 156, 159, 160, 163, 164, 169, 170, 171, 176, 177, 179, 180, 190, 208, 213, 214, 218, 219, 228, 229, 231, 234, 236, 240, 241, 243, 247, 248, 250, 251, 253, 277, 278, 289, 299, 304, 315, 318, 323, 327, 328, 332, 336, 338, 342, 345, 348, 352, 353, 356, 363, 367, 368, 369, 372, 375, 376, 378, 391, 394, 403, 406, 409, 415, 419, 420, 422, 425, 426, 427, 428, 432, 433, 435, 442, 449, 452, 466, 467, 470, 472, 475, 476, 478, 479, 483, 485, 491, 494, 498, 499, 503, 504, 511, 513, 515, 516, 526, 531, 533, 537, 540, 543, 544, 555, 561, 564, 567, 570, 573, 574, 576, 579, 589, 592, 598, 601, 603, 606, 608, 612, 613, 616, 617, 619, 621, 624, 626, 629, 634, 635, 637, 641, 643, 645, 647, 649, 650, 653, 657, 662, 666, 671, 672, 675, 677, 679, 680, 685, 686, 689, 693, 699, 702, 703, 705, 707, 710, 711, 717, 722, 724, 727, 728, 737, 739, 747, 748, 756, 758, 759, 760, 761, 762, 777, 779, 783, 788, 794, 799, 802, 803, 808, 811, 814, 822, 823, 824

[{'idx': 1, 'text': '잘 들려요?'},
 {'idx': 11, 'text': '그 혹시 내심의 정의가 뭔지 알고 있어요?'},
 {'idx': 13, 'text': '혹시 아까도 소리 들렸나요?'},
 {'idx': 17, 'text': '무슨 말인지 이해가 좀 되나요?'},
 {'idx': 22, 'text': '이해가 되나요?'},
 {'idx': 25, 'text': '무슨 말인지 이해가 되나요?'},
 {'idx': 29,
  'text': '네 그래서 그렇게 넘어가면 될 것 같고 근데 좀 궁금한 게 있는데 혹시 연준 학생은 공부할 때 아이패드를 쓰나요?'},
 {'idx': 30, 'text': '아니면 보통 종이로 쓰나요?'},
 {'idx': 31, 'text': '요즘 학생들은 그러면 다 아이패드를 많이 쓰나요?'},
 {'idx': 34, 'text': '12번에 지금 요거는 문제 자체가 어렵지는 않은데 혹시 그 도형이 닮은 분은 중학교 때 배웠죠'},
 {'idx': 36, 'text': '이게 만약에 A 대 B면은 그러면은 2차원 도형에서 넓이비는 어떻게 되죠'},
 {'idx': 41, 'text': '그러면은 A에 B의 넓이비가 어떻게 되죠?'},
 {'idx': 43, 'text': '맞죠?'},
 {'idx': 50, 'text': '지금 요 경우에는 지금 요게 원뿔이면 2차원 도형이에요?'},
 {'idx': 51, 'text': '3차원 도형이에요?'},
 {'idx': 62, 'text': '무슨 말을 이해해도 되나요?'},
 {'idx': 64,
  'text': '아까 전에 평면도형이랑 입체도형에서 표면도형 같은 경우는 제곱에 비례를 하고 입체도형 같은 경우에는 세제곱에 비례한다는 말까지는 이해가 되나요?'},
 {'idx': 65, 'text': '그러면 이 경우에는 3차원이 입체도형이니까 세제곱에 비례를 해야 된다는 것까지도 납득이 되죠?'},
 {'idx': 70, 'text': '그러면은 다이몬비가 3

In [22]:
study_idx = sorted(list(study_idx))
processed_result = [item for item in preprocessed_result if item['idx'] in study_idx]

# 중복을 허용하며 20개씩 묶는 함수
def split_with_overlap(data, chunk_size, overlap):
    result = []
    step = chunk_size - overlap  # 한 번에 이동할 크기
    for i in range(0, len(data), step):
        result.append(data[i:i + chunk_size])
        if i + chunk_size >= len(data):  # 마지막 부분은 리스트를 초과하지 않도록 처리
            break
    return result

# 예시: 20개씩 묶으면서 10개씩 중복
chunk_size = 10
overlap = 5
chunks_with_overlap = split_with_overlap(processed_result, chunk_size, overlap) 

In [23]:
chunks_with_overlap

[[{'idx': 11, 'text': '그 혹시 내심의 정의가 뭔지 알고 있어요?'},
  {'idx': 17, 'text': '무슨 말인지 이해가 좀 되나요?'},
  {'idx': 22, 'text': '이해가 되나요?'},
  {'idx': 25, 'text': '무슨 말인지 이해가 되나요?'},
  {'idx': 34, 'text': '12번에 지금 요거는 문제 자체가 어렵지는 않은데 혹시 그 도형이 닮은 분은 중학교 때 배웠죠'},
  {'idx': 36, 'text': '이게 만약에 A 대 B면은 그러면은 2차원 도형에서 넓이비는 어떻게 되죠'},
  {'idx': 41, 'text': '그러면은 A에 B의 넓이비가 어떻게 되죠?'},
  {'idx': 50, 'text': '지금 요 경우에는 지금 요게 원뿔이면 2차원 도형이에요?'},
  {'idx': 51, 'text': '3차원 도형이에요?'},
  {'idx': 64,
   'text': '아까 전에 평면도형이랑 입체도형에서 표면도형 같은 경우는 제곱에 비례를 하고 입체도형 같은 경우에는 세제곱에 비례한다는 말까지는 이해가 되나요?'}],
 [{'idx': 36, 'text': '이게 만약에 A 대 B면은 그러면은 2차원 도형에서 넓이비는 어떻게 되죠'},
  {'idx': 41, 'text': '그러면은 A에 B의 넓이비가 어떻게 되죠?'},
  {'idx': 50, 'text': '지금 요 경우에는 지금 요게 원뿔이면 2차원 도형이에요?'},
  {'idx': 51, 'text': '3차원 도형이에요?'},
  {'idx': 64,
   'text': '아까 전에 평면도형이랑 입체도형에서 표면도형 같은 경우는 제곱에 비례를 하고 입체도형 같은 경우에는 세제곱에 비례한다는 말까지는 이해가 되나요?'},
  {'idx': 65, 'text': '그러면 이 경우에는 3차원이 입체도형이니까 세제곱에 비례를 해야 된다는 것까지도 납득이 되죠?'},
  {'idx': 80, 'text': '이해가

In [24]:
from langchain_core.runnables import RunnableParallel
from prompt import digging

llm = ChatOpenAI(model='gpt-4o')

system_prompt = digging.prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
results = chain.batch([
    {"user_message": chunk} for chunk in chunks_with_overlap
])

In [25]:
results

['{\n    "idx": 36,\n    "result": true,\n    "reason": "해당 질문은 학생이 넓이비를 계산하는 방법에 대해 알고 있는지 스스로 생각해보고 대답하도록 유도하는 메타인지형 질문입니다."\n}',
 '{\n    "idx": 65,\n    "result": true,\n    "reason": "해당 질문은 학생이 3차원 입체도형의 성질에 대해 이해하고 있는지를 스스로 점검하게 하는 메타인지형 질문입니다."\n}',
 '{\n    "idx": 65,\n    "result": true,\n    "reason": "해당 질문은 학생이 3차원 도형이 세제곱에 비례해야 한다는 것을 이해하고 있는지, 그리고 그 이유를 납득하고 있는지를 묻는 메타인지형 질문입니다."\n}',
 '{\n    "idx": 156,\n    "result": true,\n    "reason": "이 질문은 학생이 닮음의 조건을 기억하고 있는지를 물어봄으로써 학생의 사고 과정을 유도하고, 학생의 이해도를 점검하는 메타인지형 질문입니다."\n}',
 '{\n    "idx": 197,\n    "reason": "해당 질문은 학생이 문제를 풀면서 이해가 안 되는 부분을 스스로 인지하고 질문하도록 유도하는 메타인지형 질문입니다."\n}',
 '{\n    "idx": 182,\n    "result": true,\n    "reason": "이 질문은 학생에게 두 변의 길이 비율을 스스로 생각해보고 이해했는지 확인하는 메타인지형 질문입니다."\n}',
 '{\n    "idx": 244,\n    "result": true,\n    "reason": "해당 질문은 학생이 자신의 이해도를 점검하고, 이해하지 못한 부분을 스스로 인식하여 답변하도록 유도하는 메타인지형 질문입니다."\n}',
 '{\n    "idx": 244,\n    "result": true,\n    "reason": "해당 질문은 학생에게 자신의 이해도를 점검하고, 스스로 어